In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [1]:
from sklearn.svm import SVR
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.impute import KNNImputer
from sklearn.model_selection import ShuffleSplit, GridSearchCV, train_test_split, cross_val_score 
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer, FunctionTransformer, KBinsDiscretizer, StandardScaler, OneHotEncoder

In [2]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 

path_train = 'train.csv' # содержит только имя файла, без имен папок !!!

In [3]:
def irq_mse(y_true, y_pred):
    delta = y_pred - y_true
    quant_75 = np.quantile(delta, 0.75)
    quant_25 = np.quantile(delta, 0.25)
    irq = quant_75 - quant_25
    mask = (delta < quant_25 - 1.5*irq) | (delta > quant_75 + 1.5*irq)
    if mask.sum() == 0:
        irq_mse = 0
    else:
        delta_new = (delta[mask])**2
        irq_mse = -1/len(delta_new) * np.sum(delta_new)
    return irq_mse

In [7]:
train = pd.read_csv(path_train)

In [9]:
print(f'всего дубликатов-строк в датасете: {train.duplicated().sum()}')
train.drop_duplicates(inplace=True)

всего дубликатов-строк в датасете: 78


In [11]:
X = train.drop(columns=['price'])
y = train['price']

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X, y, shuffle=True, random_state=42)

### трансформер на ошибки

In [15]:
cat_features = ['cut','color','clarity']
axis_features = ['x','y','z']
num_features = ['carat','depth','table']

def replace_axis(df):
    index_to_change = df['x'] < df['z']
    karman = df.loc[index_to_change, 'z']
    df.loc[index_to_change, 'z'] = df.loc[index_to_change, 'x']
    df.loc[index_to_change, 'x'] = karman
    
    index_to_change = df['y'] < df['z']
    karman = df.loc[index_to_change, 'z']
    df.loc[index_to_change, 'z'] = df.loc[index_to_change, 'y']
    df.loc[index_to_change, 'y'] = karman

    index_to_change = df['x'] < df['y']
    karman = df.loc[index_to_change, 'x']
    df.loc[index_to_change, 'x'] = df.loc[index_to_change, 'y']
    df.loc[index_to_change, 'y'] = karman
    return df


def fix_depth(df):
    df['depth'] = round(2*df['z']/(df['x'] + df['y'])*100,1)
    return df


def fix_outliers_axis(df):
    for col in axis_features:
        quant_75 = np.quantile(df[col], axis=0, q=0.75)
        quant_25 = np.quantile(df[col], axis=0, q=0.25)
        irq = quant_75 - quant_25
        mask = (df[col] < quant_25 - 1.5*irq) | (df[col] > quant_75 + 1.5*irq)
        index_of_outliers = df[mask].index
        df.loc[index_of_outliers, col] = df.apply(lambda x: x[col] / 10, axis=1)
    return df


axis_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(missing_values=0, n_neighbors=5, weights='uniform')),
    ('replace_axis', FunctionTransformer(replace_axis)),
    ('fix_depth', FunctionTransformer(fix_depth)),
    ('fix_outliers', FunctionTransformer(fix_outliers_axis)),
])


CT_for_test = ColumnTransformer([
        ("axis_transformer", axis_transformer, axis_features)],
        remainder='passthrough'
        ).set_output(transform='pandas')

display(CT_for_test)

ColumnTransformer(remainder='passthrough',
                  transformers=[('axis_transformer',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(missing_values=0)),
                                                 ('replace_axis',
                                                  FunctionTransformer(func=<function replace_axis at 0x0000029C7D9A42C0>)),
                                                 ('fix_depth',
                                                  FunctionTransformer(func=<function fix_depth at 0x0000029C7D9A44A0>)),
                                                 ('fix_outliers',
                                                  FunctionTransformer(func=<function fix_outliers_axis at 0x0000029C7D9A47C0>))]),
                                 ['x', 'y', 'z'])])

In [17]:
def fix_dataset(df):
    return CT_for_test.fit_transform(df).rename(
        columns={'axis_transformer__x':'x',
                 'axis_transformer__y':'y',
                 'axis_transformer__z':'z',
                 'remainder__carat':'carat',
                 'axis_transformer__depth':'depth',
                 'remainder__table':'table',
                 'remainder__cut':'cut',
                 'remainder__color':'color',
                 'remainder__clarity':'clarity'}).drop(columns='remainder__depth')

In [19]:
X_train = fix_dataset(X_train)
X_test = fix_dataset(X_test)

### трансформер на всякие приколы

In [21]:
scaler = StandardScaler()


axis_transformer = Pipeline(steps=[
    ('polynomials', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', scaler)
])

num_transformer = Pipeline(steps=[
    ('boxcox', QuantileTransformer(output_distribution='uniform')),
    ('scaler', scaler)
])


CT = ColumnTransformer([
        ("axis_transformer", axis_transformer, axis_features),
        ("num", num_transformer, num_features),
        ("binning", KBinsDiscretizer(n_bins=10, strategy='uniform'), ['carat']),
        ("cat", OneHotEncoder(sparse_output=False), cat_features),
        ],
        remainder='passthrough'
        )

display(CT)

ColumnTransformer(remainder='passthrough',
                  transformers=[('axis_transformer',
                                 Pipeline(steps=[('polynomials',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['x', 'y', 'z']),
                                ('num',
                                 Pipeline(steps=[('boxcox',
                                                  QuantileTransformer()),
                                                 ('scaler', StandardScaler())]),
                                 ['carat', 'depth', 'table']),
                                ('binning',
                                 KBinsDiscretizer(n_bins=10,
                                                  strategy='uniform'),
                                 ['carat']),
                                ('cat', OneHotEncoder(sparse_output=False),
                                 ['cut', 'color', 'clarity'])])

In [23]:
model_target =  TransformedTargetRegressor(
                regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator', LinearRegression())
                            ]),
                func = np.log,
                inverse_func = np.exp
)

model_target.fit(X_train, y_train)

y_pred = model_target.predict(X_train)
print(f'mean_squared_error = {mean_squared_error(y_train, y_pred)}')
print(f'irq_mse = {irq_mse(y_train, y_pred)}')

mean_squared_error = 562282.8885365746
irq_mse = -3104996.4146437007


In [25]:
model_target =  TransformedTargetRegressor(
                regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator', Ridge())
                            ]),
                func = np.log,
                inverse_func = np.exp
)

model_target.fit(X_train, y_train)

y_pred = model_target.predict(X_train)
print(f'mean_squared_error = {mean_squared_error(y_train, y_pred)}')
print(f'irq_mse = {irq_mse(y_train, y_pred)}')

mean_squared_error = 581316.2422487342
irq_mse = -3253585.6381706386


In [27]:
model_target =  TransformedTargetRegressor(
                regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator', SVR())
                            ]),
                func = np.log,
                inverse_func = np.exp
)

model_target.fit(X_train, y_train)

y_pred = model_target.predict(X_train)
print(f'mean_squared_error = {mean_squared_error(y_train, y_pred)}')
print(f'irq_mse = {irq_mse(y_train, y_pred)}')

mean_squared_error = 277613.0514705769
irq_mse = -1567778.3376819864


In [28]:
print('Test:')
y_pred = model_target.predict(X_test)
print(f'mean_squared_error = {mean_squared_error(y_test, y_pred)}')
print(f'irq_mse = {irq_mse(y_test, y_pred)}')

Test:
mean_squared_error = 409238.0164056408
irq_mse = -2421375.859327483
